In [3]:
##############################################################################
# Examples for the Minimum Correlation Algorithm paper
# Copyright (C) 2012 Michael Kapler
#
# Forecast-Free Algorithms: A New Benchmark For Tactical Strategies
# http://cssanalytics.wordpress.com/2011/08/09/forecast-free-algorithms-a-new-benchmarkfor-tactical-strategies/
#
# For more information please visit my blog at www.SystematicInvestor.wordpress.com
# or drop me a line at TheSystematicInvestor at gmail
###############################################################################
###############################################################################
# Main program to run all examples
#
# Please Load Systematic Investor Toolbox prior to running this routine
###############################################################################
# Load Systematic Investor Toolbox (SIT)
# http://systematicinvestor.wordpress.com/systematic-investor-toolbox/
###############################################################################
# setInternet2(TRUE)
con = gzcon(url('http://www.systematicportfolio.com/sit.gz', 'rb'))
# con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

In [4]:
min.corr.paper.examples <- function()
{
#*****************************************************************
# Load historical data sets
#******************************************************************
load.packages('quantmod')
#*****************************************************************
# Load historical data for Futures and Forex
#******************************************************************
data <- new.env()
getSymbols.TB(env = data, auto.assign = T, download = T)
bt.prep(data, align='remove.na', dates='1990::')
save(data,file='FuturesForex.Rdata')
#*****************************************************************
# Load historical data for ETFs
#******************************************************************
tickers = spl('SPY,QQQ,EEM,IWM,EFA,TLT,IYR,GLD')
data <- new.env()
getSymbols(tickers, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T)
for(i in ls(data)) data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)
# TLT first date is 7/31/2002
bt.prep(data, align='keep.all', dates='2002:08::')
save(data,file='ETF.Rdata')
#*****************************************************************
# Load historical data for dow stock (engle)
#******************************************************************
load.packages('quantmod,quadprog')
tickers = spl('AA,AXP,BA,CAT,DD,DIS,GE,IBM,IP,JNJ,JPM,KO,MCD,MMM,MO,MRK,MSFT')
data <- new.env()
getSymbols(tickers, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T)
for(i in ls(data)) data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)
bt.prep(data, align='keep.all', dates='1980::')
save(data,file='Dow.Engle.Rdata')
#*****************************************************************
# Load historical data for ETFs
#******************************************************************
load.packages('quantmod,quadprog')
tickers = spl('VTI,IEV,EEM,EWJ,AGG,GSG,GLD,ICF')
data <- new.env()
getSymbols(tickers, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T)
for(i in ls(data)) data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)
bt.prep(data, align='keep.all', dates='2003:10::')
save(data,file='ETF2.Rdata')
#*****************************************************************
# Load historical data for nasdaq 100 stocks
#******************************************************************
load.packages('quantmod,quadprog')
#tickers = nasdaq.100.components()
tickers =
spl('ATVI,ADBE,ALTR,AMZN,AMGN,APOL,AAPL,AMAT,ADSK,ADP,BBBY,BIIB,BMC,BRCM,CHRW,CA,CELG,CEP
H,CERN,CHKP,CTAS,CSCO,CTXS,CTSH,CMCSA,COST,DELL,XRAY,DISH,EBAY,ERTS,EXPD,ESRX,FAST,FISV,F
LEX,FLIR,FWLT,GILD,HSIC,HOLX,INFY,INTC,INTU,JBHT,KLAC,LRCX,LIFE,LLTC,LOGI,MAT,MXIM,MCHP,M
SFT,MYL,NTAP,NWSA,NVDA,ORLY,ORCL,PCAR,PDCO,PAYX,PCLN,QGEN,QCOM,RIMM,ROST,SNDK,SIAL,SPLS,S
BUX,SRCL,SYMC,TEVA,URBN,VRSN,VRTX,VOD,XLNX,YHOO')
data <- new.env()
for(i in tickers) {
try(getSymbols(i, src = 'yahoo', from = '1980-01-01', env = data, auto.assign =
T), TRUE)
data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)
}
bt.prep(data, align='keep.all', dates='1995::')
save(data,file='nasdaq.100.Rdata')
#*****************************************************************
# Run all strategies
#******************************************************************
names = spl('ETF,FuturesForex,Dow.Engle,ETF2,nasdaq.100')
lookback.len = 60
periodicitys = spl('weeks,months')
periodicity = periodicitys[1]
prefix = paste(substr(periodicity,1,1), '.', sep='')
for(name in names) {
load(file = paste(name, '.Rdata', sep=''))
obj = portfolio.backtest.helper(data$prices, periodicity, lookback.len =
lookback.len, prefix = prefix,
min.risk.fns =
'min.corr.portfolio,max.div.portfolio,min.var.portfolio,equal.risk.portfolio,equal.weight
.portfolio')
save(obj, file=paste(name, lookback.len, periodicity, '.bt', '.Rdata', sep=''))
}
#*****************************************************************
# Create Reports
#******************************************************************
for(name in names) {
load(file=paste(name, '.Rdata', sep=''))
# create summary of inputs report
custom.input.report.helper(paste('report.', name, sep=''), data)
# create summary of strategies report
load(file=paste(name, lookback.len, periodicity, '.bt', '.Rdata', sep=''))
custom.report.helper(paste('report.', name, lookback.len, periodicity, sep=''),
create.strategies(obj, data))
}
#*****************************************************************
# Futures and Forex: rescale strategies to match Equal Weight strategy risk profile
#******************************************************************
names = spl('FuturesForex')
for(name in names) {
load(file=paste(name, '.Rdata', sep=''))
# create summary of strategies report
load(file=paste(name, lookback.len, periodicity, '.bt', '.Rdata', sep=''))
leverage = c(4, 5, 20, 20, 3, 1)
custom.report.helper(paste('report.leverage.', name, lookback.len, periodicity,
sep=''),
create.strategies(obj, data, leverage), T)
}
}

In [5]:
###############################################################################
# Portfolio Construction and Optimization routines
###############################################################################
#*****************************************************************
# Weighting Schemes
#*****************************************************************
equal.weight.portfolio <- function
(
ia, # input assumptions
constraints # constraints
)
{
rep(1/ia$n, ia$n)
}
equal.risk.portfolio <- function
(
ia, # input assumptions
constraints # constraints
)
{
x = 1 / sqrt( diag(ia$cov) )
# normalize weights to sum up to 1
return( x / sum(x) )
}
min.var.portfolio <- function
(
ia, # input assumptions
constraints # constraints
)
{
sol = solve.QP(Dmat=ia$cov, dvec=rep(0, ia$n),
Amat=constraints$A, bvec=constraints$b, meq=constraints$meq)
x = sol$solution
return( x )
}
# Toward Maximum Diversification by Y. Choueifaty, Y. Coignard
# The Journal of Portfolio Management, Fall 2008, Vol. 35, No. 1: pp. 40-51
max.div.portfolio <- function
(
ia, # input assumptions
constraints # constraints
)
{
sol = solve.QP(Dmat=ia$correlation, dvec=rep(0, ia$n),
Amat=constraints$A, bvec=constraints$b, meq=constraints$meq)
x = sol$solution
# re-scale weights to penalize for risk
x = x / sqrt( diag(ia$cov) )
# normalize weights to sum up to 1
return( x / sum(x) )
}
# MinCorr by David Varadi
min.corr.portfolio <- function
(
ia, # input assumptions
constraints # constraints
)
{
upper.index = upper.tri(ia$correlation)
cor.m = ia$correlation[upper.index]
cor.mu = mean(cor.m)
cor.sd = sd(cor.m)
norm.dist.m = 0 * ia$correlation
diag(norm.dist.m) = NA
norm.dist.m[upper.index] = 1-pnorm(cor.m, cor.mu, cor.sd)
norm.dist.m = (norm.dist.m + t(norm.dist.m))
norm.dist.avg = rowMeans(norm.dist.m, na.rm=T)
norm.dist.rank = rank(-norm.dist.avg)
norm.dist.weight = norm.dist.rank / sum(norm.dist.rank)
diag(norm.dist.m) = 0
weighted.norm.dist.average = norm.dist.weight %*% norm.dist.m
final.weight = weighted.norm.dist.average / sum(weighted.norm.dist.average)
# re-scale weights to penalize for risk
x = final.weight
x = x / sqrt( diag(ia$cov) )
# normalize weights to sum up to 1
return( x / sum(x) )
}
# MinCorr2 by David Varadi
min.corr2.portfolio <- function
(
ia, # input assumptions
constraints # constraints
)
{
cor.m = ia$correlation
diag(cor.m) = 0
avg = rowMeans(cor.m)
cor.mu = mean(avg)
cor.sd = sd(avg)
norm.dist.avg = 1-pnorm(avg, cor.mu, cor.sd)
norm.dist.rank = rank(-norm.dist.avg)
norm.dist.weight = norm.dist.rank / sum(norm.dist.rank)
weighted.norm.dist.average = norm.dist.weight %*% (1-cor.m)
final.weight = weighted.norm.dist.average / sum(weighted.norm.dist.average)
# re-scale weights to penalize for risk
x = final.weight
x = x / sqrt( diag(ia$cov) )
# normalize weights to sum up to 1
return( x / sum(x) )
}
#*****************************************************************
# Portfolio Backtest helper function
#*****************************************************************
portfolio.backtest.helper <- function
(
prices, # prices
periodicity = 'weeks', # rebalancing frequency
period.ends = endpoints(prices, periodicity), # rebalancing times
lookback.len = 60, # lookback to construct input assumptions each period
prefix = '',
universe = prices[period.ends,]>0,
min.risk.fns = 'min.var.portfolio' # portfolio construction functions
)
{
load.packages('quadprog,corpcor')
#*****************************************************************
# Setup
#*****************************************************************
period.ends = period.ends[period.ends > 0]
universe[is.na(universe)] = F
if(is.character(min.risk.fns)) min.risk.fns = spl(min.risk.fns)
#*****************************************************************
# Code Strategies
#******************************************************************
prices = coredata(prices)
ret = prices / mlag(prices) - 1
start.i = which(period.ends >= (lookback.len + 1))[1]
weight = NA * prices[period.ends,]
weight[] = 0
weights = list()
for(f in min.risk.fns)
weights[[ paste(prefix, gsub('\\.portfolio', '', f), sep='') ]] = weight
risk.contributions = list()
for(w in names(weights))
risk.contributions[[w]] = NA * weights[[w]]
# construct portfolios
for( j in start.i:len(period.ends) ) {
i = period.ends[j]
# histtory to construct input assumptions
hist = ret[ (i- lookback.len +1):i, ]
# require all assets to have full price history
include.index = count(hist)== lookback.len
index = universe[j,] & include.index
n = sum(index)
if(n > 0) {
hist = hist[ , index]
# 0 <= x.i <= 1
constraints = new.constraints(n, lb = 0, ub = 1)
constraints = add.constraints(diag(n), type='>=', b=0, constraints)
constraints = add.constraints(diag(n), type='<=', b=1, constraints)
# SUM x.i = 1
constraints = add.constraints(rep(1, n), 1, type = '=', constraints)
# create historical input assumptions
ia = list()
ia$n = n
s0 = apply(hist, 2, sd)
ia$correlation = cor(hist, use='complete.obs', method='pearson')
ia$cov = ia$correlation * (s0 %*% t(s0))
ia$cov = make.positive.definite(ia$cov, 0.000000001)
ia$correlation = make.positive.definite(ia$correlation, 0.000000001)
# find optimal portfolios under different risk measures
for(f in min.risk.fns) {
weights[[ paste(prefix, gsub('\\.portfolio', '', f),sep='') ]][j,index] =
match.fun(f)(ia, constraints)
}
# compute risk contributions implied by portfolio weihgts
for(w in names(weights))
risk.contributions[[ w ]][j, index] =
portfolio.risk.contribution(weights[[ w ]][j, index], ia)
}
if( j %% 10 == 0) cat(j, '\n')
}
return(list(weights = weights, period.ends = period.ends,
risk.contributions = risk.contributions,
periodicity = periodicity, lookback.len = lookback.len))
}
###############################################################################
# Custom Report routines
###############################################################################
#*****************************************************************
# Create summary of inputs report
#*****************************************************************
custom.input.report.helper <- function(filename, data) {
filename.pdf = paste(filename, '.pdf', sep='')
filename.csv = paste(filename, '.csv', sep='')
#*****************************************************************
# Create Report
#******************************************************************
# put all reports into one pdf file
pdf(file = filename.pdf, width=8.5, height=11)
# Input Details
layout(1:2)
asset.models = list()
for(i in data$symbolnames) {
data$weight[] = NA
data$weight[,i] = 1
asset.models[[i]] = bt.run(data, silent=T)
}
asset.summary = plotbt.strategy.sidebyside(asset.models, return.table=T)
# plot correlations
ret.log = bt.apply.matrix(data$prices, ROC, type='continuous')
temp = compute.cor(ret.log, 'pearson')
temp[] = plota.format(100 * temp, 0, '', '%')
plot.table(temp, smain='Correlation', highlight = TRUE, colorbar = TRUE)
# line plot for each input series
layout(matrix(1:4,2,2))
if( is.null(data$symbol.groups) ) {
index = order(data$symbolnames)
for(i in data$symbolnames[index])
plota(data[[i]], type='l', cex.main=0.7,main= i)
} else {
index = order(data$symbol.groups)
for(i in data$symbolnames[index])
plota(data[[i]], type='l', cex.main=0.7, main= paste(i,
data$symbol.groups[i], data$symbol.descriptions.print[i], sep=' / ') )
asset.summary = rbind(data$symbol.groups, data$symbol.descriptions.print,
asset.summary)
}
# close pdf
dev.off()
#*****************************************************************
# save summary & equity curves into csv file
#******************************************************************
load.packages('abind')
write.csv(asset.summary, filename.csv)
cat('\n\n', file=filename.csv, append=TRUE)
write.table(temp, sep=',', row.names = , col.names = NA,
file=filename.csv, append=TRUE)
}
#*****************************************************************
# Create summary of strategies report
#*****************************************************************
custom.report.helper <- function(filename, obj) {
filename.pdf = paste(filename, '.pdf', sep='')
filename.csv = paste(filename, '.csv', sep='')
models = obj$models
#*****************************************************************
# Create Report
#******************************************************************
# put all reports into one pdf file
pdf(file = filename.pdf, width=8.5, height=11)
# Plot perfromance
plotbt(models, plotX = T, log = 'y', LeftMargin = 3)
mtext('Cumulative Performance', side = 2, line = 1)
# Plot Strategy Statistics Side by Side
out = plotbt.strategy.sidebyside(models, perfromance.fn = 'custom.returns.kpi',
return.table=T)
# Bar chart in descending order of the best algo by Sharpe Ratio, CAGR, Gini,
Herfindahl
layout(mat=matrix(1:4, 2, 2, byrow=FALSE))
par(mar=c(4, 3, 2, 2))
col = spl('lightgray,red')
custom.order = c(T,T,F,F,F)
names(custom.order) = spl('Sharpe,Cagr,RC Gini,RC Herfindahl,Volatility')
for(i in names(custom.order)) {
y = as.double(out[i,])
index = order(y, decreasing = custom.order[i])
x = barplot(y[index], names.arg = '',
col=iif(y[index] > 0, col[1], col[2]),
main=i,
border = 'darkgray',las=2)
text(x, 0 * x, colnames(out)[index], adj=c(-0.1,1), srt=90, xpd = TRUE)
grid(NA,NULL)
abline(h=0, col='black')
# add best worst labels
mtext('worst', side = 1,line = 0, outer = F, adj = 1, font = 1, cex = 1)
mtext('best', side = 1,line = 0, outer = F, adj = 0, font = 1, cex = 1)
}
# Plot Portfolio Turnover for each strategy
#layout(1)
y = sapply(models, compute.turnover, data)
index = order(y, decreasing = F)
barplot.with.labels(y[index], 'Average Annual Portfolio Turnover', plotX = F,
label='both')
# add best worst labels
mtext('worst', side = 1,line = 0, outer = F, adj = 1, font = 1, cex = 1)
mtext('best', side = 1,line = 0, outer = F, adj = 0, font = 1, cex = 1)
# summary allocation statistics for each model
custom.summary.positions(obj$weights)
# monhtly returns for each model
custom.period.chart(models)
# Plot transition maps
layout(1:len(models))
for(m in names(models)) {
plotbt.transition.map(models[[m]]$weight, name=m)
legend('topright', legend = m, bty = 'n')
}
# Plot transition maps for Risk Contributions
dates = index(models[[1]]$weight)[obj$period.ends]
layout(1:len(models))
for(m in names(models)) {
plotbt.transition.map(make.xts(obj$risk.contributions[[m]], dates),
name=paste('Risk Contributions',m))
legend('topright', legend = m, bty = 'n')
}
# close pdf
dev.off()
#*****************************************************************
# save summary & equity curves into csv file
#******************************************************************
load.packages('abind')
write.csv(out, filename.csv)
cat('\n\n', file=filename.csv, append=TRUE)
out = abind(lapply(models, function(m) m$equity))
colnames(out) = names(models)
write.xts(make.xts(out, index(models[[1]]$equity)), filename.csv, append=TRUE)
}
#*****************************************************************
# Create strategies based on portfolio weights
#*****************************************************************
create.strategies <- function
(
obj, # portfolio.backtest object:list(weights=weights,period.ends=period.ends)
data, # historical prices
leverage = 1
)
{
# adjust weights
if(len(leverage) == 1) leverage = rep(leverage, len(obj$weights))
names(leverage) = names(obj$weights)
for(i in names(obj$weights)) obj$weights[[i]] = leverage[i] * obj$weights[[i]]
#*****************************************************************
# Create strategies
#******************************************************************
models = list()
for(i in names(obj$weights)) {
data$weight[] = NA
data$weight[obj$period.ends,] = obj$weights[[i]]
models[[i]] = bt.run.share(data, clean.signal = F)
models[[i]]$risk.contribution = obj$risk.contributions[[i]]
models[[i]]$period.weight = obj$weights[[i]]
}
obj$models = models
return(obj)
}
#*****************************************************************
# Summary Positions for each model
#*****************************************************************
custom.summary.positions <- function(weights) {
layout(1:len(weights))
for(w in names(weights)) {
tickers = colnames(weights[[w]])
n = len(tickers)
temp = matrix(NA, nr=4, nc=n)
colnames(temp) = tickers
rownames(temp) = spl('Avg Pos,Max Pos,Min Pos,# Periods')
temp['Avg Pos',] = 100 * apply(weights[[w]],2,mean,na.rm=T)
temp['Max Pos',] = 100 * apply(weights[[w]],2,max,na.rm=T)
temp['Min Pos',] = 100 * apply(weights[[w]],2,min,na.rm=T)
temp['# Periods',] = apply(weights[[w]] > 1/1000,2,sum,na.rm=T)
temp[] = plota.format(temp, 0, '', '')
plot.table(temp, smain=w)
}
}
#*****************************************************************
# Helper function to create barplot
#*****************************************************************
custom.profit.chart <- function(data, main, cols) {
par(mar=c(4, 3, 2, 2),cex.main=2,cex.sub=1, cex.axis=1.5,cex.lab=1.5)
barplot(data, names.arg = names(data),
col=iif(data > 0, cols[1], cols[2]),
main=main,
cex.names = 1.5, border = 'darkgray',las=2)
grid(NA,NULL)
abline(h=0,col='black')
abline(h=mean(data),col='gray',lty='dashed',lwd=3)
}
#*****************************************************************
# Monhtly returns for each model
#*****************************************************************
custom.period.chart <- function(models) {
for(imodel in 1:len(models)) {
equity = models[[imodel]]$equity
#*****************************************************************
# Compute monthly returns
#******************************************************************
period.ends = endpoints(equity, 'months')
period.ends = unique(c(1, period.ends[period.ends > 0]))
ret = equity[period.ends,] / mlag(equity[period.ends,]) - 1
ret = ret[-1]
ret.by.month = create.monthly.table(ret)
ret.by.month = 100 * apply(ret.by.month, 2, mean, na.rm=T)
#*****************************************************************
# Compute annual returns
#******************************************************************
period.ends = endpoints(equity, 'years')
period.ends = unique(c(1, period.ends[period.ends > 0]))
ret = equity[period.ends,] / mlag(equity[period.ends,]) - 1
ret.by.year = ret[-1]
#*****************************************************************
# Create plots
#******************************************************************
# create layout
ilayout =
'1,1
2,2
2,2
2,2
2,2
2,2
3,4
3,4
3,4
3,4
3,4
3,4'
plota.layout(ilayout)
# make dummy table with name of strategy
make.table(1,1)
a = matrix(names(models)[imodel],1,1)
cex = plot.table.helper.auto.adjust.cex(a)
draw.cell(a[1],1,1, text.cex=cex,frame.cell=F)
# plots
temp = plotbt.monthly.table(equity)
# plot months
cols = spl('green,red')
custom.profit.chart(ret.by.month, 'Average Monthly Returns', cols)
# plot years
ret = 100*as.vector(ret.by.year)
names(ret) = date.year(index(ret.by.year))
custom.profit.chart(ret, 'Annual Returns', cols)
}
}
#*****************************************************************
# Custom Summary function to add consentration statistics (Gini and Herfindahl)
#
# On the properties of equally-weighted risk contributions portfolios by
# S. Maillardy, T. Roncalliz, J. Teiletchex (2009)
# A.4 Concentration and turnover statistics, page 22
#*****************************************************************
custom.returns.kpi <- function
(
bt, # backtest object
trade.summary = NULL
)
{
out = list()
w = bt$period.weight
rc = bt$risk.contribution
# Average Number of Holdings
out[[ 'Avg #' ]] = mean(rowSums(w > 1/1000)) / 100
# Consentration stats
out[[ 'W Gini' ]] = mean(portfolio.concentration.gini.coefficient(w), na.rm=T)
out[[ 'W Herfindahl' ]] = mean(portfolio.concentration.herfindahl.index(w), na.rm=T)
# Consentration stats on marginal risk contributions
out[[ 'RC Gini' ]] = mean(portfolio.concentration.gini.coefficient(rc), na.rm=T)
out[[ 'RC Herfindahl' ]] = mean(portfolio.concentration.herfindahl.index(rc),
na.rm=T)
out = lapply(out, function(x) if(is.double(x)) round(100*x,1) else x)
out = c(bt.detail.summary(bt)$System, out)
return( list(System=out))
}

In [7]:
###############################################################################
# Load Systematic Investor Toolbox (SIT)
###############################################################################
# setInternet2(TRUE)
con = gzcon(url('http://www.systematicportfolio.com/sit.gz', 'rb'))
    source(con)
close(con)

# Load supporting R code for Minimum Correlation Algorithm Paper
con=url('http://www.systematicportfolio.com/mincorr_paper.r')
    source(con)
close(con)

#*****************************************************************
# Load historical data sets
#****************************************************************** 
    load.packages('quantmod')   
    
    #*****************************************************************
    # Load historical data for Futures and Forex
    #****************************************************************** 
    data <- new.env()
    getSymbols.TB(env = data, auto.assign = T, download = T)
        
    bt.prep(data, align='remove.na', dates='1990::')
    save(data,file='FuturesForex.Rdata')
    

    #*****************************************************************
    # Load historical data for ETFs
    #****************************************************************** 
    tickers = spl('SPY,QQQ,EEM,IWM,EFA,TLT,IYR,GLD')

    data <- new.env()
    getSymbols(tickers, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T)
        for(i in ls(data)) data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)                            
    # TLT first date is 7/31/2002
    bt.prep(data, align='keep.all', dates='2002:08::')
    save(data,file='ETF.Rdata')

    
    #*****************************************************************
    # Load historical data for dow stock (engle)
    #****************************************************************** 
    load.packages('quantmod,quadprog')
    tickers = spl('AA,AXP,BA,CAT,DD,DIS,GE,IBM,IP,JNJ,JPM,KO,MCD,MMM,MO,MRK,MSFT')
        
    data <- new.env()
    getSymbols(tickers, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T)
        for(i in ls(data)) data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)                            
        
    bt.prep(data, align='keep.all', dates='1980::')
    save(data,file='Dow.Engle.Rdata')


    #*****************************************************************
    # Load historical data for ETFs
    #****************************************************************** 
    load.packages('quantmod,quadprog')
    tickers = spl('VTI,IEV,EEM,EWJ,AGG,GSG,GLD,ICF')
        
    data <- new.env()
    getSymbols(tickers, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T)
        for(i in ls(data)) data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)                            
        
    bt.prep(data, align='keep.all', dates='2003:10::')  
    save(data,file='ETF2.Rdata')

    
    #*****************************************************************
    # Load historical data for nasdaq 100 stocks
    #****************************************************************** 
    load.packages('quantmod,quadprog')
    #tickers = nasdaq.100.components()
    tickers = spl('ATVI,ADBE,ALTR,AMZN,AMGN,APOL,AAPL,AMAT,ADSK,ADP,BBBY,BIIB,BMC,BRCM,CHRW,CA,CELG,CEPH,CERN,CHKP,CTAS,CSCO,CTXS,CTSH,CMCSA,COST,DELL,XRAY,DISH,EBAY,ERTS,EXPD,ESRX,FAST,FISV,FLEX,FLIR,FWLT,GILD,HSIC,HOLX,INFY,INTC,INTU,JBHT,KLAC,LRCX,LIFE,LLTC,LOGI,MAT,MXIM,MCHP,MSFT,MYL,NTAP,NWSA,NVDA,ORLY,ORCL,PCAR,PDCO,PAYX,PCLN,QGEN,QCOM,RIMM,ROST,SNDK,SIAL,SPLS,SBUX,SRCL,SYMC,TEVA,URBN,VRSN,VRTX,VOD,XLNX,YHOO')
        
    data <- new.env()
    for(i in tickers) {
        try(getSymbols(i, src = 'yahoo', from = '1980-01-01', env = data, auto.assign = T), TRUE)
        data[[i]] = adjustOHLC(data[[i]], use.Adjusted=T)                           
    }
    bt.prep(data, align='keep.all', dates='1995::')
    save(data,file='nasdaq.100.Rdata')


    
    
#*****************************************************************
# Run all strategies
#****************************************************************** 
    names = spl('ETF,FuturesForex,Dow.Engle,ETF2,nasdaq.100')   
    lookback.len = 60
    periodicitys = spl('weeks,months')
    periodicity = periodicitys[1]
    prefix = paste(substr(periodicity,1,1), '.', sep='')
    
    
    
    for(name in names) {
        load(file = paste(name, '.Rdata', sep=''))
        
        obj = portfolio.allocation.helper(data$prices, periodicity, lookback.len = lookback.len, prefix = prefix,
            min.risk.fns = 'min.corr.portfolio,min.corr2.portfolio,max.div.portfolio,min.var.portfolio,risk.parity.portfolio,equal.weight.portfolio',
            custom.stats.fn = 'portfolio.allocation.custom.stats')      
            
        save(obj, file=paste(name, lookback.len, periodicity, '.bt', '.Rdata', sep=''))
    }
        
    
    #*****************************************************************
    # Create Reports
    #****************************************************************** 
    for(name in names) {
        load(file=paste(name, '.Rdata', sep=''))
        
        # create summary of inputs report
        custom.input.report.helper(paste('report.', name, sep=''), data)
        
        # create summary of strategies report
        load(file=paste(name, lookback.len, periodicity, '.bt', '.Rdata', sep=''))
        custom.report.helper(paste('report.', name, lookback.len, periodicity, sep=''), 
            create.strategies(obj, data))   
    }

    
    #*****************************************************************
    # Futures and Forex: rescale strategies to match Equal Weight strategy risk profile
    #****************************************************************** 
    names = spl('FuturesForex') 
    for(name in names) {
        load(file=paste(name, '.Rdata', sep=''))

        # create summary of strategies report
        load(file=paste(name, lookback.len, periodicity, '.bt', '.Rdata', sep=''))
            leverage = c(5, 4, 15, 20, 3, 1)
        custom.report.helper(paste('report.leverage.', name, lookback.len, periodicity, sep=''), 
            create.strategies(obj, data, leverage)) 
    }



Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Attaching package: 'TTR'

The following object is masked _by_ '.GlobalEnv':

    DVI

Version 0.4-0 included new data defaults. See ?getSymbols.
Warning message in download.file("http://www.tradingblox.com/Data/DataOnly.zip", :
"cannot open URL 'http://www.tradingblox.com/Data/DataOnly.zip': HTTP status was '404 Not Found'"

ERROR: Error in download.file("http://www.tradingblox.com/Data/DataOnly.zip", : cannot open URL 'http://www.tradingblox.com/Data/DataOnly.zip'


In [8]:
s = 'ABCDE'

In [10]:
import re

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: import re
           ^
